In [12]:
from __future__ import print_function
import os
import sys
import math
import pickle
import boto3
import os
import numpy as np
import kg
import pandas as pd
# from tqdm import tqdm
import time
import argparse
import logging
import re
# tqdm.pandas()
# pandarallel.initialize(progress_bar=True)
# bucket = os.environ.get("BUCKET_NAME", " ")
# raw_data_folder = os.environ.get("RAW_DATA", " ")
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# tqdm_notebook().pandas()
s3client = boto3.client('s3')

In [13]:
########################################
# 从s3同步数据
########################################
def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))


def write_to_s3(filename, bucket, key):
    print("upload s3://{}/{}".format(bucket, key))
    with open(filename, 'rb') as f:  # Read in binary mode
        # return s3client.upload_fileobj(f, bucket, key)
        return s3client.put_object(
            ACL='bucket-owner-full-control',
            Bucket=bucket,
            Key=key,
            Body=f
        )

def write_str_to_s3(content, bucket, key):
    print("write s3://{}/{}, content={}".format(bucket, key, content))
    s3client.put_object(Body=str(content).encode("utf8"), Bucket=bucket, Key=key, ACL='bucket-owner-full-control')

default_bucket = 'aws-gcr-rs-sol-demo-ap-southeast-1-522244679887'
default_prefix = 'sample-data'
parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str)
parser.add_argument('--prefix', type=str)
args, _ = parser.parse_known_args()
bucket = args.bucket
prefix = args.prefix

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

out_s3_path = "s3://{}/{}/feature/content/inverted-list".format(bucket, prefix)

local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)

bucket=aws-gcr-rs-sol-demo-ap-southeast-1-522244679887
prefix='sample-data'


In [14]:
# prepare model for batch process
meta_file_prefix = "{}/model/meta_files".format(prefix)
os.environ['GRAPH_BUCKET'] = bucket
os.environ['KG_DBPEDIA_KEY'] = '{}/kg_dbpedia.txt'.format(meta_file_prefix)
os.environ['KG_ENTITY_KEY'] = '{}/entities_dbpedia.dict'.format(meta_file_prefix)
os.environ['KG_RELATION_KEY'] = '{}/relations_dbpedia.dict'.format(meta_file_prefix)
os.environ['KG_DBPEDIA_TRAIN_KEY'] = '{}/kg_dbpedia_train.txt'.format(meta_file_prefix)
os.environ['KG_ENTITY_TRAIN_KEY'] = '{}/entities_dbpedia_train.dict'.format(meta_file_prefix)
os.environ['KG_RELATION_TRAIN_KEY'] = '{}/relations_dbpedia_train.dict'.format(meta_file_prefix)
os.environ['KG_ENTITY_INDUSTRY_KEY'] = '{}/entity_industry.txt'.format(meta_file_prefix)
os.environ['KG_VOCAB_KEY'] = '{}/vocab.json'.format(meta_file_prefix)
os.environ['DATA_INPUT_KEY'] = ''
os.environ['TRAIN_OUTPUT_KEY'] = '{}/model/rank/content/dkn_embedding_latest/'.format(prefix)

kg_path = os.environ['GRAPH_BUCKET']
dbpedia_key = os.environ['KG_DBPEDIA_KEY']
entity_key = os.environ['KG_ENTITY_KEY']
relation_key = os.environ['KG_RELATION_KEY']
dbpedia_train_key = os.environ['KG_DBPEDIA_TRAIN_KEY']
entity_train_key = os.environ['KG_ENTITY_TRAIN_KEY']
relation_train_key = os.environ['KG_RELATION_TRAIN_KEY']
entity_industry_key = os.environ['KG_ENTITY_INDUSTRY_KEY']
vocab_key = os.environ['KG_VOCAB_KEY']
data_input_key = os.environ['DATA_INPUT_KEY']
train_output_key = os.environ['TRAIN_OUTPUT_KEY']

env = {
    'GRAPH_BUCKET': kg_path,
    'KG_DBPEDIA_KEY': dbpedia_key,
    'KG_ENTITY_KEY': entity_key,
    'KG_RELATION_KEY': relation_key,
    'KG_DBPEDIA_TRAIN_KEY': dbpedia_train_key,
    'KG_ENTITY_TRAIN_KEY': entity_train_key,
    'KG_RELATION_TRAIN_KEY': relation_train_key,
    'KG_ENTITY_INDUSTRY_KEY': entity_industry_key,
    'KG_VOCAB_KEY': vocab_key,
    'DATA_INPUT_KEY': data_input_key,
    'TRAIN_OUTPUT_KEY': train_output_key
}

print("Kg env: {}".format(env))
graph = kg.Kg(env)  # Where we keep the model when it's loaded

Kg env: {'GRAPH_BUCKET': 'aws-gcr-rs-sol-demo-ap-southeast-1-522244679887', 'KG_DBPEDIA_KEY': 'sample-data/model/meta_files/kg_dbpedia.txt', 'KG_ENTITY_KEY': 'sample-data/model/meta_files/entities_dbpedia.dict', 'KG_RELATION_KEY': 'sample-data/model/meta_files/relations_dbpedia.dict', 'KG_DBPEDIA_TRAIN_KEY': 'sample-data/model/meta_files/kg_dbpedia_train.txt', 'KG_ENTITY_TRAIN_KEY': 'sample-data/model/meta_files/entities_dbpedia_train.dict', 'KG_RELATION_TRAIN_KEY': 'sample-data/model/meta_files/relations_dbpedia_train.dict', 'KG_ENTITY_INDUSTRY_KEY': 'sample-data/model/meta_files/entity_industry.txt', 'KG_VOCAB_KEY': 'sample-data/model/meta_files/vocab.json', 'DATA_INPUT_KEY': '', 'TRAIN_OUTPUT_KEY': 'sample-data/model/rank/content/dkn_embedding_latest/'}
load file: aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/meta_files/kg_dbpedia_train.txt
load file: aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/meta_files/kg_dbpedia.txt
load file: aws-gcr-rs

In [15]:
graph.train(max_step=200)

Reading train triples....
Finished. Read 8879 train triples.
|Train|: 8879
Total initialize time 0.032 seconds
[proc 0][Train](2/200) average pos_loss: 3.386800765991211
[proc 0][Train](2/200) average neg_loss: 0.7110021561384201
[proc 0][Train](2/200) average loss: 2.0489014983177185
[proc 0][Train](2/200) average regularization: 6.5461050553494715e-06
[proc 0][Train] 2 steps take 0.127 seconds
[proc 0]sample: 0.019, forward: 0.055, backward: 0.047, update: 0.005
[proc 0][Train](4/200) average pos_loss: 7.592857599258423
[proc 0][Train](4/200) average neg_loss: 2.809683084487915
[proc 0][Train](4/200) average loss: 5.201270341873169
[proc 0][Train](4/200) average regularization: 1.5214416634989902e-05
[proc 0][Train] 2 steps take 0.110 seconds
[proc 0]sample: 0.003, forward: 0.054, backward: 0.048, update: 0.005
[proc 0][Train](6/200) average pos_loss: 9.213049411773682
[proc 0][Train](6/200) average neg_loss: 3.259446620941162
[proc 0][Train](6/200) average loss: 6.236248016357422
[p

[proc 0][Train](50/200) average pos_loss: 1.187422513961792
[proc 0][Train](50/200) average neg_loss: 0.7786665856838226
[proc 0][Train](50/200) average loss: 0.9830445647239685
[proc 0][Train](50/200) average regularization: 2.4372593543375842e-05
[proc 0][Train] 2 steps take 0.106 seconds
[proc 0]sample: 0.022, forward: 0.040, backward: 0.039, update: 0.005
[proc 0][Train](52/200) average pos_loss: 1.2930764555931091
[proc 0][Train](52/200) average neg_loss: 1.2186270654201508
[proc 0][Train](52/200) average loss: 1.2558517456054688
[proc 0][Train](52/200) average regularization: 2.2518186597153544e-05
[proc 0][Train] 2 steps take 0.086 seconds
[proc 0]sample: 0.002, forward: 0.040, backward: 0.039, update: 0.005
[proc 0][Train](54/200) average pos_loss: 1.3550967574119568
[proc 0][Train](54/200) average neg_loss: 0.6699808537960052
[proc 0][Train](54/200) average loss: 1.0125387907028198
[proc 0][Train](54/200) average regularization: 2.4348044462385587e-05
[proc 0][Train] 2 steps t

[proc 0][Train](98/200) average pos_loss: 0.9956911504268646
[proc 0][Train](98/200) average neg_loss: 1.0556930601596832
[proc 0][Train](98/200) average loss: 1.025692105293274
[proc 0][Train](98/200) average regularization: 3.6676843592431396e-05
[proc 0][Train] 2 steps take 0.104 seconds
[proc 0]sample: 0.021, forward: 0.040, backward: 0.038, update: 0.004
[proc 0][Train](100/200) average pos_loss: 0.9932705760002136
[proc 0][Train](100/200) average neg_loss: 0.97077876329422
[proc 0][Train](100/200) average loss: 0.9820246994495392
[proc 0][Train](100/200) average regularization: 3.613175795180723e-05
[proc 0][Train] 2 steps take 0.086 seconds
[proc 0]sample: 0.003, forward: 0.039, backward: 0.038, update: 0.004
[proc 0][Train](102/200) average pos_loss: 1.1053794026374817
[proc 0][Train](102/200) average neg_loss: 0.9585698395967484
[proc 0][Train](102/200) average loss: 1.0319746136665344
[proc 0][Train](102/200) average regularization: 3.6359615478431806e-05
[proc 0][Train] 2 st

[proc 0][Train](144/200) average pos_loss: 1.0280828475952148
[proc 0][Train](144/200) average neg_loss: 0.9132647812366486
[proc 0][Train](144/200) average loss: 0.9706738591194153
[proc 0][Train](144/200) average regularization: 3.8489401049446315e-05
[proc 0][Train] 2 steps take 0.086 seconds
[proc 0]sample: 0.003, forward: 0.040, backward: 0.038, update: 0.004
[proc 0][Train](146/200) average pos_loss: 0.7690311968326569
[proc 0][Train](146/200) average neg_loss: 0.9190304577350616
[proc 0][Train](146/200) average loss: 0.8440308272838593
[proc 0][Train](146/200) average regularization: 3.880084659613203e-05
[proc 0][Train] 2 steps take 0.115 seconds
[proc 0]sample: 0.030, forward: 0.041, backward: 0.039, update: 0.004
[proc 0][Train](148/200) average pos_loss: 1.006872832775116
[proc 0][Train](148/200) average neg_loss: 0.8479791805148125
[proc 0][Train](148/200) average loss: 0.9274260401725769
[proc 0][Train](148/200) average regularization: 3.9886273953015916e-05
[proc 0][Train

[proc 0][Train](190/200) average pos_loss: 0.9436261653900146
[proc 0][Train](190/200) average neg_loss: 0.8250166773796082
[proc 0][Train](190/200) average loss: 0.8843214213848114
[proc 0][Train](190/200) average regularization: 4.217925379634835e-05
[proc 0][Train] 2 steps take 0.086 seconds
[proc 0]sample: 0.003, forward: 0.039, backward: 0.038, update: 0.004
[proc 0][Train](192/200) average pos_loss: 0.8933379650115967
[proc 0][Train](192/200) average neg_loss: 0.8443978428840637
[proc 0][Train](192/200) average loss: 0.8688679039478302
[proc 0][Train](192/200) average regularization: 4.115147021366283e-05
[proc 0][Train] 2 steps take 0.084 seconds
[proc 0]sample: 0.002, forward: 0.039, backward: 0.038, update: 0.004
[proc 0][Train](194/200) average pos_loss: 0.737901896238327
[proc 0][Train](194/200) average neg_loss: 0.7619196474552155
[proc 0][Train](194/200) average loss: 0.7499107718467712
[proc 0][Train](194/200) average regularization: 4.2059069528477266e-05
[proc 0][Train]

In [16]:
import numpy as np
entity_embs = np.load('/home/ec2-user/workplace/recommender-system-solution/src/offline/news/model-update-graph/sample-data/model/rank/content/dkn_embedding_latest/aws-gcr-rs-sol-demo-ap-southeast-1-522244679887_RotatE_entity.npy')

In [17]:
len(entity_embs)

7680

In [5]:
# !aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_word_embedding.npy
# !aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_entity_embedding.npy

In [6]:
n = 0
for k, v in entities_dbpedia_train.items():
    print("k {} v {}".format(k,v))
    if n > 10:
        break
    n = n + 1

NameError: name 'entities_dbpedia_train' is not defined